In [1]:
atom_string = """<?xml version="1.0" encoding="utf-8"?>

<feed xmlns="http://www.w3.org/2005/Atom">

	<title>Example Feed</title>
	<subtitle>A subtitle.</subtitle>
	<id>urn:uuid:60a76c80-d399-11d9-b91C-0003939e0af6</id>
	<updated>2003-12-13T18:30:02Z</updated>
	
	<entry>
		<title>Atom-Powered Robots Run Amok</title>
		<link rel="about" href="http://dbpedia.org/resource/Kumquat"/>
		<id>urn:uuid:1225c695-cfb8-4ebb-aaaa-80da344efa6a</id>
		<updated>2003-12-13T18:30:02Z</updated>
		<summary>Some text.</summary>
		<content type="xhtml">
			<div xmlns="http://www.w3.org/1999/xhtml">
				<p>This is the entry content.</p>
			</div>
		</content>
		<author>
			<name>Ross Fenning</name>
			<uri>http://rossfenning.co.uk/</uri>
		</author>
	</entry>

</feed>"""

import feedparser
feed = feedparser.parse(atom_string)

In [2]:
from rdflib import Graph, URIRef, Namespace, Literal, BNode
ATOM = Namespace('http://www.w3.org/2005/Atom/')


def atom2rdf(atom_feed):
    g = Graph()
    g.bind('atom', ATOM)

    for entry in atom_feed.entries:
        subject = URIRef(entry.id)
        g.add((subject, ATOM.title, Literal(entry.title)))
        g.add((subject, ATOM.summary, Literal(entry.summary)))

        for author in entry.authors:
            a = BNode()
            g.add((subject, ATOM.author, a))
            print(author)
            for prop in ['name', 'email']:
                if prop in author:
                    g.add((a, getattr(ATOM, prop), Literal(author[prop])))
            if 'href' in author:
                g.add((a, ATOM.url, URIRef(author.href)))

#         for link in entry.links:
#             print(link)
#             g.add((subject, link.rel, link.href))
            
    return g
    

In [3]:
    
rdf_graph = atom2rdf(feed)
    
print(rdf_graph.serialize(format='turtle').decode('utf-8'))

{'name': 'Ross Fenning', 'href': 'http://rossfenning.co.uk/'}
@prefix atom: <http://www.w3.org/2005/Atom/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:uuid:1225c695-cfb8-4ebb-aaaa-80da344efa6a> atom:author [ atom:name "Ross Fenning" ;
            atom:url <http://rossfenning.co.uk/> ] ;
    atom:summary "Some text." ;
    atom:title "Atom-Powered Robots Run Amok" .




In [4]:
foaf_string = '''
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

<http://rossfenning.co.uk/>
    a foaf:Person ;
    foaf:name "Ross Fenning" ;
    foaf:interest <http://dbpedia.org/resource/Linked_Data> ;
    foaf:interest <http://dbpedia.org/resource/Wine> ;
    foaf:interest <http://dbpedia.org/resource/Python> .
'''

another_rdf_graph = Graph()
another_rdf_graph.parse(data=foaf_string, format='turtle')
print(another_rdf_graph.serialize(format='turtle').decode('utf-8'))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://rossfenning.co.uk/> a foaf:Person ;
    foaf:interest <http://dbpedia.org/resource/Linked_Data>,
        <http://dbpedia.org/resource/Python>,
        <http://dbpedia.org/resource/Wine> ;
    foaf:name "Ross Fenning" .




In [5]:
combined_graph = rdf_graph + another_rdf_graph

print(combined_graph.serialize(format='turtle').decode('utf-8'))

@prefix atom: <http://www.w3.org/2005/Atom/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:uuid:1225c695-cfb8-4ebb-aaaa-80da344efa6a> atom:author [ atom:name "Ross Fenning" ;
            atom:url <http://rossfenning.co.uk/> ] ;
    atom:summary "Some text." ;
    atom:title "Atom-Powered Robots Run Amok" .

<http://rossfenning.co.uk/> a foaf:Person ;
    foaf:interest <http://dbpedia.org/resource/Linked_Data>,
        <http://dbpedia.org/resource/Python>,
        <http://dbpedia.org/resource/Wine> ;
    foaf:name "Ross Fenning" .




In [7]:
from FuXi.Rete.RuleStore import RuleStore, SetupRuleStore


def enrich(graph, rules):
    rule_store, rule_graph, network = SetupRuleStore(makeNetwork=True)
    closure_delta = Graph()

    network.inferredFacts = closure_delta

    for rule in rules:
        network.buildNetworkFromClause(rule)

    network.feedFactsToAdd(generateTokenSet(graph))
    return graph + closure_delta

rule_store = N3RuleStore()
namespace_manager = NamespaceManager(Graph(rule_store))
rule_graph = Graph(rule_store, namespace_manager=namespace_manager)
rule_graph.parse(format='n3', data='''
@prefix atom: <http://www.w3.org/2005/Atom/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

{
?entry atom:author ?author .
?author atom:url ?url .
?author atom:name ?name .
?url a foaf:Person .
} => {
?entry atom
}

''')


ModuleNotFoundError: No module named 'FuXi'